In [0]:
import dlt
from pyspark.sql.functions import lit, concat_ws, row_number
from pyspark.sql.window import Window

def int_ar_receipt_integrate():
    return dlt.read("int_ar_receipt_integrate")

@dlt.table(
    name="trn_ar_receipt_transform"
)
def trn_ar_receipt_transform():
    df = dlt.read("int_ar_receipt_integrate")
    # Sort DataFrame
    sorted_df = df.orderBy("receipt_number")
    # Window specification for row_number
    windowSpec = Window.orderBy("receipt_number")
    df_seq = sorted_df.withColumn("seq_num", row_number().over(windowSpec))
    # Add default value column
    df_seq = df_seq.withColumn("Record_Type", lit("customer"))
    # Add concatenated column
    df_seq = df_seq.withColumn("Unique_column", concat_ws("_", df_seq.customer_name, df_seq.receipt_number))
    return df_seq